In [1]:
import pandas as pd 
import numpy  as np 

data = {'id'        : [1,2,1,2,3,1,3,1,3],
        'pay_month' : [1,1,2,2,2,3,3,4,4],
        'salary'    : [20,20,30,30,40,40,60,60,70]
        }

employee = pd.DataFrame(data)

In [2]:
employee

,id,pay_month,salary
0,1,1,20
1,2,1,20
2,1,2,30
3,2,2,30
4,3,2,40
5,1,3,40
6,3,3,60
7,1,4,60
8,3,4,70


In [3]:
# using a copy for the second option
employee2 = employee.copy()

In [6]:
employee['cummulative_salary'] = employee.groupby('id').agg({'salary':np.cumsum})
employee['ranks']              = employee.groupby('id')['pay_month'].rank(method = 'first', ascending = False)
employee[employee.ranks > 1.0][['id','pay_month','salary','cummulative_salary']].sort_values(by = 'id')

,id,pay_month,salary,cummulative_salary
0,1,1,20,20
2,1,2,30,50
5,1,3,40,90
1,2,1,20,20
4,3,2,40,40
6,3,3,60,100


**Here's another weird option using not np.cumsum method**

In [7]:
distinct = employee2.drop_duplicates('pay_month')['pay_month'].copy()
distinct 

0    1
2    2
5    3
7    4
Name: pay_month, dtype: int64

In [8]:
month       = []
cummulative = []
rows = [v for v in distinct]
for i in rows:
    for j in range(i):
        month.append(i)
        cummulative.append(j + 1)

In [9]:
cummulative = pd.DataFrame(cummulative)
cummulative.rename(columns = {0:'cummulative_pay_month'}, inplace = True)
month       = pd.DataFrame(month)
month.rename(columns = {0:'pay_month'}, inplace = True)
df          = pd.concat([month,cummulative], axis = 1)
df

,pay_month,cummulative_pay_month
0,1,1
1,2,1
2,2,2
3,3,1
4,3,2
5,3,3
6,4,1
7,4,2
8,4,3
9,4,4


In [10]:
df =pd.merge(employee2, df, on = 'pay_month', how = 'left')
df

,id,pay_month,salary,cummulative_pay_month
0,1,1,20,1
1,2,1,20,1
2,1,2,30,1
3,1,2,30,2
4,2,2,30,1
5,2,2,30,2
6,3,2,40,1
7,3,2,40,2
8,1,3,40,1
9,1,3,40,2


In [11]:
df1 = employee2.copy()
df1.rename(columns = {'pay_month':'cummulative_pay_month', 'salary':'cummulative_salary'}, inplace = True)
df1

,id,cummulative_pay_month,cummulative_salary
0,1,1,20
1,2,1,20
2,1,2,30
3,2,2,30
4,3,2,40
5,1,3,40
6,3,3,60
7,1,4,60
8,3,4,70


In [12]:
df = pd.merge(df, df1, on = ['id','cummulative_pay_month'], how = 'left')
df

,id,pay_month,salary,cummulative_pay_month,cummulative_salary
0,1,1,20,1,20.0
1,2,1,20,1,20.0
2,1,2,30,1,20.0
3,1,2,30,2,30.0
4,2,2,30,1,20.0
5,2,2,30,2,30.0
6,3,2,40,1,NaN
7,3,2,40,2,40.0
8,1,3,40,1,20.0
9,1,3,40,2,30.0


In [13]:
df['cummulative_salary'] = df['cummulative_salary'].replace(np.nan, 0)
df.drop(['cummulative_pay_month'], axis = 1, inplace = True)
df

,id,pay_month,salary,cummulative_salary
0,1,1,20,20.0
1,2,1,20,20.0
2,1,2,30,20.0
3,1,2,30,30.0
4,2,2,30,20.0
5,2,2,30,30.0
6,3,2,40,0.0
7,3,2,40,40.0
8,1,3,40,20.0
9,1,3,40,30.0


In [14]:
df = df.groupby(['id','pay_month','salary'])['cummulative_salary'].sum()

In [17]:
df.reset_index()
df

id  pay_month  salary
1   1          20         20.0
    2          30         50.0
    3          40         90.0
    4          60        150.0
2   1          20         20.0
    2          30         50.0
3   2          40         40.0
    3          60        100.0
    4          70        170.0
Name: cummulative_salary, dtype: float64